In [1]:
# Initial Imports
import pandas as pd
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

# Import S&P 500 stock ticker list
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df_stocks = table[0]
df_stocks.head()
df_stocks.to_csv('S&P500-Info.csv')
df_stocks.to_csv('S&P500-Symbols.csv', columns=['Symbol'])
df_tickers = df_stocks['Symbol']
df_tickers.head()

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
load_dotenv()

In [ ]:
# Import historical price performance for S&P 500 stocks over the last 7 years
timeframe = '1Day'
start_date = pd.Timestamp('2016-06-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2023-04-30', tz='America/New_York').isoformat()

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

alpaca = tradeapi.REST(
   alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

tickers = df_stocks['Symbol'][:300, 400].tolist()

df_sp500_hist = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

In [ ]:
df_sp500_hist.head()

In [ ]:
# test test tes